# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最终可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: ** 这是一个分类问题，因为我们需要的并不是一个连续的值，而是离散的值。 最终，结果将会是 set(需要帮助的学生) 和 set(不需要帮助的学生) 其中一个。

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [1]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score


# 载入学生数据集
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [3]:
# TODO： 计算学生的数量
n_students = len(student_data)

# TODO： 计算特征数量
n_features = len(student_data.columns)-1

# TODO： 计算通过的学生数
n_passed = len(student_data.where(student_data['passed'] == 'yes').dropna())

# TODO： 计算未通过的学生数
n_failed = len(student_data.where(student_data['passed'] == 'no').dropna())

# TODO： 计算通过率
grad_rate = 100 * n_passed / float(n_students)

# 输出结果
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为**大多数的机器学习算法都会期望输入数字特征进行计算。**

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [4]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [5]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)
    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        # 收集转换后的列
        output = output.join(col_data)
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约76%），使用95个数据点作为测试集（约24%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [6]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.model_selection import train_test_split

# TODO：设置训练集的数量
num_train = 300

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train

# TODO：把数据集混洗和分割成上面定义的训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, train_size=num_train, random_state=12)

# 显示分割的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: ** 在

- Naive-Bayes

    - 应用场景：最著名的应用是使用于“垃圾邮件过滤”。
    
    - 优势：只需要根据少量的训练数据估计出必要的参数(变量的均值和方差),学习和预测的效率高。对小规模的数据集友好，能够处理多分类任务，适合增量式学习，尤其是数据集的大小超过了内存的容纳空间时，可以分批次训练。对于缺失的数据不太敏感，在这种情况下，可以使用拉普拉斯修正方法规避错误。通过极大似然估计和贝叶斯定理，只需要花费线性时间。
    
    - 缺点：朴素贝叶斯算法假设数据集的特征之间是条件独立的，以此为基础构建概率模型，是一种生成学习方法。因为前提是假设了数据特征之间是条件独立的，但是现实数据集中这一点并不是一定会成立的，所以在这种理想的假设下，训练而得到的分类器的分类性能并不一定能够得到保证。对输入数据的表达形式很敏感。即便如此，在实际运行过程中，还是具有良好的表现。
    
    - 为什么使用这个模型：当前数据集较小，可以使用朴素贝叶斯算法进行分类。NB算法对大多数数据集都有好的表现。

- SVM Classifier

    - 应用场景：用于文本和超文本的分类，在归纳和直推方法中都可以显著减少所需要的有类标的样本数。（因为文本文件中的特征非常多，纬度高，使用SVM是个好的选择）
    
    - 优势：训练出来的模型具有很高的精确度，即使数据集并非线性可分的，通过选择合适的kernel方法也可以在基特征空间中有很好的表现。SVM尽量保持与样本间距离的性质导致它抗攻击的能力更强。不考虑数据是否非线性，可以用于分类和回归。对于分类问题，使用SVM试水看效果是个不错的选择。
    
    - 缺点：内存密集，对调参和核函数的选择敏感。SVM仅直接适用于两分类问题，对于多分类问题，必须先将其变换到两分类问题。对于训练出来的模型，其参数难以理解。
    
    - 为什么使用这个模型：该数据集的特征多大48个，维度较高，二分类问题。使用SVM是个不错的选择。

- K-Nearest-Neighbors

    - 应用场景：文件分类
    
    - 优势：基本理论简单易于理解，并不需要显式地进行模型训练。既可用于分类，也可以用于回归。不考虑是否线性可分的问题。对数据集的分布情况并不做任何假设，准确度高，对异常值不敏感。特点是完全跟着数据集在走，没有数学模型可言。
    
    - 缺点：高方差低偏差，容易过拟合。对于K值的选择较为复杂。kd树的构建和查找过程较为复杂。当样本的分布不平衡的时候，效果并不理想。计算复杂度高，空间复杂度高。对数据的局部结构非常敏感。
    
    - 为什么使用这个模型：当前情况对于数据集内部结构并不了解。KNN算法可解释性强，对于数据的分布不做任何假设，准确度高，可以使用该算法看效果。


## 参考资料
- 《统计学习方法》-------------------李航
- 《机器学习》----------------------周志华
-  维基百科
-  知乎
-  Quora

### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    # 输出分类器名称和训练集大小
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # 训练一个分类器
    train_classifier(clf, X_train, y_train)
    
    # 输出训练和测试的预测结果
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### 练习: 模型评价指标

借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** **这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。**
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [11]:
# TODO：从sklearn中引入三个监督学习模型
from sklearn import naive_bayes
from sklearn import svm
from sklearn import neighbors

# TODO：设置训练集大小
X_train_100 = X_train.head(100).copy()
y_train_100 = y_train.head(100).copy()

X_train_200 = X_train.head(200).copy()
y_train_200 = y_train.head(200).copy()

X_train_300 = X_train
y_train_300 = y_train

In [12]:
# 模型一：Naive-Bayes Classifier
nb = naive_bayes.GaussianNB()
train_predict(nb, X_train_100, y_train_100, X_test, y_test)
train_predict(nb, X_train_200, y_train_200, X_test, y_test)
train_predict(nb, X_train_300, y_train_300, X_test, y_test)
print "===================================================="

# 模型二：Support-Vector-Machine Classifier
svc = svm.SVC(random_state=13)
train_predict(svc, X_train_100, y_train_100, X_test, y_test)
train_predict(svc, X_train_200, y_train_200, X_test, y_test)
train_predict(svc, X_train_300, y_train_300, X_test, y_test)
print "===================================================="

# 模型三：K-Nearest-Neighbor Classifier
knn = neighbors.KNeighborsClassifier()
train_predict(knn, X_train_100, y_train_100, X_test, y_test)
train_predict(knn, X_train_200, y_train_200, X_test, y_test)
train_predict(knn, X_train_300, y_train_300, X_test, y_test)
print "===================================================="

# classifiers = [nb, svc, knn]
# for clf in classifiers:
#     train_predict(clf, X_train_100, y_train_100, X_test, y_test)
#     train_predict(clf, X_train_200, y_train_200, X_test, y_test)
#     train_predict(clf, X_train_300, y_train_300, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0050 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8276.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7586.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0030 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8162.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7040.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0030 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8107.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7244.
Training a SVC using a training set size of 100. . .
Trained model in 0.0120 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8841.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7619.
Training a SVC using a training set size of 200. . .
Trained model in 0.0070 seconds
Ma

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - Naive-Bayes Classifier **  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :------: | :-----: | :-----------: | :--------: | :--------: |
| 100      |0.0030   |0.0010         |0.8276      |0.7586      |
| 200      |0.0040   |0.0010         |0.8162      |0.7040      |
| 300      |0.0020   |0.0010         |0.8107      |0.7244      |

** 分类器 2 - SVM Classifier **  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :------: | :-----: | :-----------: | :--------: | :--------: |
| 100      |0.0120   |0.0020         |0.8841      |0.7619      |
| 200      |0.0070   |0.0030         |0.8837      |0.8000      |
| 300      |0.0210   |0.0030         |0.8761      |0.8000      |

** 分类器 3 - K-Nearest-Neighbors Classifier**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :------: | :-----: | :-----------: | :--------: | :--------: |
| 100      |0.0020   |0.0070         |0.8209      |0.7538      |
| 200      |0.0020   |0.0040         |0.8235      |0.7770      |
| 300      |0.0020   |0.0070         |0.8447      |0.8082      |                   

## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: ** 在此处，我选择SVM分类器作为最佳模型。
由上述表格，我们得以的到以下结论：
1. SVM分类器在不同数据集下的平均训练时间最长，但其在训练集上的表现是最好的，在测试集上的平均F1参数也是最好的。证明该模型对于该数据集的拟合是上述上个模型中最佳的。
2. 虽说训练时间相对而言长了点，但是预测时间却是很快的啊。

### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **
    我们使用的是一种叫做“支持向量机”的数学方法来解决的这个问题。现在我们希望可以把这组学生根据是否需要学习帮助给分成两个不同的部分。
    将学生分组的过程，就相当于把一幅扑克牌扔到空中，从中筛选出红色牌，剩下的就是黑色牌这样一个过程。这样子，我们就完成了对于学生的分组。
    我们已经具有了很多关于每个学生的生活信息，这些信息各有不同，我们试图去寻找学生信息之间是否存在某种关联性，比如说它们的家庭情况和学习成绩之间的关系，在这个过程中，通过一种叫做“核方法”的数学概念来实现这个目的。
    在将学生分组的过程中，我们希望的是学生分组具有很高的可信度，也就是说，如果新来了一个学生，这个新学生也可以正确性很高地被分类，那么就需要把这两个分组给尽可能隔离开来，形成一个很大的间隔，就像中国象棋一样，红棋和黑棋之间隔着楚河汉界，我们希望这个问题里的楚河汉界尽可能的宽。

### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [15]:
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.model_selection import GridSearchCV

# 创建调整的参数列表
parameters = {"kernel": ['linear', 'rbf', 'poly', 'sigmoid'], "C": [50,75,100], "gamma":[3, 5, 7, "auto"], "degree":[2, 3]}

# 创建需要被调整的模型
svc = svm.SVC()

# 创建一个f1评分函数
# f1_scorer = make_scorer(f1_score, pos_label="yes")
f1_scorer = make_scorer(fbeta_score, beta=1, pos_label='yes')

# 网格搜索
grid_obj = GridSearchCV(svc, parameters, f1_scorer)
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0120 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0040 seconds.
Tuned model has a testing F1 score of 0.8125.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: ** 最终模型的训练集F1值是1.0000, 测试集上的F1值是0.8125。
相对于之前的值，该值有微幅提高，提高了0.0125。

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  